In [2]:
from time import perf_counter
import numpy as np,os
import pandas as pd
from collections import defaultdict
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import BalancedBaggingClassifier, RUSBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks, RepeatedEditedNearestNeighbours
from imblearn.pipeline import Pipeline

from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score, precision_score, recall_score
import seaborn as sns,matplotlib.pyplot as plt

from library.utils import evaluate, read_data
from library.cleaners import kDN, ih_prob,FilteringEstimator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Notes
+ Need to take best value among samplers, since some don't completely balance, and IPF sucks there

In [3]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone
class IPF(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator, n=5, max_iter = 3,random_state=None):
        self.estimator = estimator
        self.n = n
        self.max_iter = max_iter
        self.random_state = random_state
        
    def clean(self,X,Y, sample_weight):
        Xt,Yt = shuffle(X,Y)
        orig_size = len(X)
        n_iters_with_small_change = 0
        tmp = 0
        while n_iters_with_small_change<self.max_iter:
            tmp += 1
            cur_size = len(Xt)
            breaks = [(len(Xt)//self.n)*i for i in range(1,self.n)]
            Xs,Ys = np.split(Xt,breaks),np.split(Yt,breaks)
            
            clfs = []
            for i in range(self.n):
                c = DecisionTreeClassifier(max_depth=2).fit(Xs[i],Ys[i])
                clfs.append(c)

            preds = np.zeros((len(Xt),self.n))
            for i in range(self.n):
                preds[:,i] = clfs[i].predict(Xt)
            eqs = preds==Yt.reshape(-1,1)  # Shape: (len(Xt),self.n)
            clean_idx = eqs.sum(axis=1)>=(self.n/2)  # Idx of clean samples
            
            try:
                sample_weight = sample_weight[clean_idx]
            except:
                pass
            Xt,Yt = Xt[clean_idx],Yt[clean_idx]
            
            cur_change = cur_size - len(Xt)
            if cur_change<=.01*orig_size:
                n_iters_with_small_change += 1
            else:
                n_iters_with_small_change = 0  #Because these small change has to be consecutively 3 times
            #print(tmp,cur_change,orig_size,cur_change/orig_size)  
        return Xt,Yt,sample_weight


    def fit(self, X, Y,sample_weight=None):
        Xf,Yf,sample_weight = self.clean(X, Y, sample_weight)
        a,b = np.unique(Y,return_counts=True)[1],np.unique(Yf,return_counts=True)[1]
        #print(a.max()/a.min(),b.max()/b.min(),a,b,Xf.shape,len(Xf)/len(X))
        assert len(np.unique(Yf))==2,"Pos class completely filtered out"
        try:
            self.estimator = self.estimator.fit(Xf, Yf,sample_weight=sample_weight)
        except TypeError as e:
            self.estimator = self.estimator.fit(Xf, Yf)
        return self

    @property
    def classes_(self):
        return self.estimator.classes_

    def predict(self, X):
        return self.estimator.predict(X)
    
    def predict_proba(self, X):
        return self.estimator.predict_proba(X)

In [5]:
imbs = {
    'smote': SMOTE(k_neighbors=5),
    'rus': RandomUnderSampler('not minority'),
    'wilson':EditedNearestNeighbours(n_neighbors=5,kind_sel='all'),  #Default was 3
    'tomek': TomekLinks(),
    'None': 'passthrough',
}
clfs = {
    'dt': DecisionTreeClassifier(max_depth=20),
    'lr': LogisticRegression(solver='lbfgs',max_iter=1000),
    'nb': GaussianNB(),
    'svm': SVC(probability=True),
    'knn': KNeighborsClassifier(n_neighbors=5),
    'rf': RandomForestClassifier(n_estimators=50),
}
bal_nb = IPF(GaussianNB())
bal_dt_bag = IPF(DecisionTreeClassifier(max_depth=20,max_features='sqrt'))
bal_dt_boost = IPF(DecisionTreeClassifier(max_depth=10))
ensembles = {
    'rboost_DT': RUSBoostClassifier(base_estimator=clone(bal_dt_boost),algorithm='SAMME',n_estimators=50),
    'rboost_NB': RUSBoostClassifier(base_estimator=clone(bal_nb),algorithm='SAMME',n_estimators=50),
    'bbag_DT': BalancedBaggingClassifier(base_estimator=clone(bal_dt_bag),n_estimators=50),
    'bbag_NB': BalancedBaggingClassifier(base_estimator=clone(bal_nb),n_estimators=50),
}

In [6]:
models = {}
for im,samp in imbs.items():
    for c,clf in clfs.items():
        models[(im,c)] = Pipeline([('samp',samp),('clf',IPF(clf))])

for m,ens in ensembles.items():
    models[('ens',m)] = ens
    
models.keys(),len(models)

(dict_keys([('smote', 'dt'), ('smote', 'lr'), ('smote', 'nb'), ('smote', 'svm'), ('smote', 'knn'), ('smote', 'rf'), ('rus', 'dt'), ('rus', 'lr'), ('rus', 'nb'), ('rus', 'svm'), ('rus', 'knn'), ('rus', 'rf'), ('wilson', 'dt'), ('wilson', 'lr'), ('wilson', 'nb'), ('wilson', 'svm'), ('wilson', 'knn'), ('wilson', 'rf'), ('tomek', 'dt'), ('tomek', 'lr'), ('tomek', 'nb'), ('tomek', 'svm'), ('tomek', 'knn'), ('tomek', 'rf'), ('None', 'dt'), ('None', 'lr'), ('None', 'nb'), ('None', 'svm'), ('None', 'knn'), ('None', 'rf'), ('ens', 'rboost_DT'), ('ens', 'rboost_NB'), ('ens', 'bbag_DT'), ('ens', 'bbag_NB')]),
 34)

In [7]:
cv = RepeatedStratifiedKFold(n_splits=5,n_repeats=5,random_state=99)
def pr_rec_score(y,yp):
    prec, rec, _ = precision_recall_curve(y,yp)
    return auc(rec,prec)
scorers = [matthews_corrcoef,pr_rec_score]

In [8]:
cols = pd.MultiIndex.from_product([imbs.keys(),clfs.keys(),[f.__name__ for f in scorers]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)
#df = pd.read_csv("IPF.csv",header=[0,1,2],index_col=0)

In [9]:
for d in DATASETS:
    X,y_noisy,y_real = read_data(d,stats=True)
    if df.loc[d,:].isna().sum()==0:
        print(f"SKIPPING {d}\n")
        continue
    for k in models:
        print(k)
        sd = perf_counter()
        r = evaluate(models[k],X,y_noisy,y_real,cv,scorers)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = r[f].mean()
        print(round(perf_counter()-sd,2),[round(r[f].mean(),3) for f in r])
    print()
    df.to_csv("IPF.csv")

activemq-5.8.0.csv noise:0.058, imb:15.847,203,3217, Shape:(3420, 65)
('smote', 'dt')
2.63 [0.235, 0.386]
('smote', 'lr')
7.32 [0.248, 0.293]
('smote', 'nb')
2.53 [0.265, 0.419]
('smote', 'svm')
63.8 [0.244, 0.205]
('smote', 'knn')
4.52 [0.231, 0.346]
('smote', 'rf')
8.33 [0.245, 0.261]
('rus', 'dt')
0.44 [0.234, 0.474]
('rus', 'lr')
0.74 [0.246, 0.325]
('rus', 'nb')
0.46 [0.271, 0.474]
('rus', 'svm')
0.6 [0.251, 0.236]
('rus', 'knn')
0.68 [0.249, 0.445]
('rus', 'rf')
1.65 [0.24, 0.393]
('wilson', 'dt')
6.44 [0.252, 0.388]
('wilson', 'lr')
7.34 [0.278, 0.342]
('wilson', 'nb')
6.55 [0.299, 0.336]
('wilson', 'svm')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


8.65 [0.257, 0.246]
('wilson', 'knn')
8.36 [0.266, 0.378]
('wilson', 'rf')
8.43 [0.259, 0.34]
('tomek', 'dt')
ERROR!!! Pos class completely filtered out
ERROR!!! Pos class completely filtered out
5.74 [0.216, 0.436]
('tomek', 'lr')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


7.18 [0.203, 0.336]
('tomek', 'nb')
ERROR!!! Pos class completely filtered out


...

ERROR!!! Pos class completely filtered out
ERROR!!! Pos class completely filtered out
ERROR!!! Pos class completely filtered out
ERROR!!! Pos class completely filtered out
ERROR!!! Pos class completely filtered out
ERROR!!! Pos class completely filtered out
ERROR!!! Pos class completely filtered out


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


ERROR!!! Pos class completely filtered out
ERROR!!! Pos class completely filtered out
0.92 [0.045, 0.525]
('ens', 'rboost_DT')


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


1.37 [0.442, 0.608]
('ens', 'rboost_NB')
1.47 [0.415, 0.597]
('ens', 'bbag_DT')
ERROR!!! Pos class completely filtered out
20.59 [0.434, 0.609]
('ens', 'bbag_NB')
ERROR!!! Pos class completely filtered out
ERROR!!! Pos class completely filtered out
20.75 [0.43, 0.596]

